In [1]:
# instalar la libreria PULP, Pyomo, Gurobt
from pulp import *

In [2]:
# sets 
refineria = ["i1", "i2", "i3"] #refineria de tipo i
cds = ["j1", "j2", "j3", "j4"] # Centros de distribución j


In [3]:
# Parametros 

capacidad_i = {"i1": 2500, "i2": 3000, "i3": 4000}# Capacidad de la refineria i
dem_max_j = {"j1": 4000, "j2": 5000, "j3": 6000, "j4": 9000} # Demanda maxima cd j

costo_add_i = {"i1": 0.125, "i2": 0.14, "i3": 0.17}

matriz_beneficio_i_j = {("i1","j1"):1.6,("i1","j2"):2,("i1", "j3"): 1.2,("i1", "j4"):1.3,
                        ("i2","j1"):1.2, ("i2","j2"):1.6, ("i2","j3"): 1.5, ("i2", "j4"):1.8,
                        ("i3", "j1"): 2.5, ("i3","j2"): 1.8, ("i3","j3"): 1.65, ("i3","j4"):1}


In [4]:
#Variable de decisión 
x = LpVariable.dicts("x", ((i, j) for i in refineria for j in cds), lowBound=0)

y = LpVariable.dicts("y", (i for i in refineria), lowBound=0)



In [5]:
#Problemas PL
barriles_enviados = LpProblem("barriles_enviados", LpMaximize)

In [6]:
# Función objetivo

barriles_enviados += lpSum(x[i,j]*matriz_beneficio_i_j[i,j] for i in refineria for j in cds)- lpSum(y[i]*costo_add_i[i] for i in refineria)


In [7]:
#Restricciones 
#Restricciones ded capacidad 

for i in refineria:
    barriles_enviados += lpSum(x[i,j] for j in cds)<= capacidad_i[i] + y[i]

#Restricciones de demanda minima 
for j in cds:
    barriles_enviados += lpSum(x[i,j] for i in refineria)>= dem_max_j[j]*0.8

# Restriccion de demanda maxima
for j in cds:
    barriles_enviados += lpSum(x[i,j] for i in refineria)<= dem_max_j[j]

# Restricciones de presupuesto 

barriles_enviados += lpSum(y[i]*costo_add_i[i] for i in refineria)<= 1500

# Restriccion del 25%

for i in refineria:
    barriles_enviados += lpSum(x[i,j] for j in cds)>= 0.25*lpSum(x[i,j] for i in refineria for j in cds)

In [8]:
# Resolver y mostrar resultados 
barriles_enviados.solve()

print("Estado:", LpStatus[barriles_enviados.status])




for i in barriles_enviados.variables():
    print(i.name, "_",i.varValue)

print(barriles_enviados.objective)

Estado: Optimal
x_('i1',_'j1') _ 0.0
x_('i1',_'j2') _ 4260.8696
x_('i1',_'j3') _ 804.34783
x_('i1',_'j4') _ 0.0
x_('i2',_'j1') _ 0.0
x_('i2',_'j2') _ 0.0
x_('i2',_'j3') _ 2930.4348
x_('i2',_'j4') _ 7200.0
x_('i3',_'j1') _ 4000.0
x_('i3',_'j2') _ 0.0
x_('i3',_'j3') _ 1065.2174
x_('i3',_'j4') _ 0.0
y_i1 _ 2565.2174
y_i2 _ 7130.4348
y_i3 _ 1065.2174
1.6*x_('i1',_'j1') + 2*x_('i1',_'j2') + 1.2*x_('i1',_'j3') + 1.3*x_('i1',_'j4') + 1.2*x_('i2',_'j1') + 1.6*x_('i2',_'j2') + 1.5*x_('i2',_'j3') + 1.8*x_('i2',_'j4') + 2.5*x_('i3',_'j1') + 1.8*x_('i3',_'j2') + 1.65*x_('i3',_'j3') + x_('i3',_'j4') - 0.125*y_i1 - 0.14*y_i2 - 0.17*y_i3
